## Setup

In [24]:
import pandas as pd
import numpy as np
import json
import glob
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 999
pd.options.display.max_columns=100
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = [15, 10]

dataDir = '/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/'
jsonDir = '/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/JSONFiles/'

## Read and fix data

In [25]:
utils = pd.read_csv(dataDir + 'eia_data/eia8602018/utility_2018.csv',header=1).fillna(0)
utils.columns = [x.lower().replace(" ","_").replace("?","") for x in utils.columns]

In [26]:
utils['owner_of_plants_reported_on_form'].value_counts()

Y    4400
0     639
Name: owner_of_plants_reported_on_form, dtype: int64

In [27]:
utils[utils['owner_of_plants_reported_on_form']==0];

In [5]:
gen_data = pd.read_csv(dataDir + 'eia_data/eia8602018/gen_2018.csv')
gen_data.drop(['Unnamed: 0'],axis=1,inplace=True)
gen_data = gen_data[~gen_data['Utility ID'].str.contains('NOTE', na=False)]
gen_data['Utility ID'] = pd.to_numeric(gen_data['Utility ID'])
gen_data.columns = gen_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('?','')

/Users/rachelanderson/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,16,39,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Now I want to merge the utility data with the generator data so I can do some sick analyses bro

I'll start by merging the utility list with the list of generators (all in 2018), so I'll see if any columns will conflict

In [6]:
[print(x) for x in gen_data.columns if x in utils.columns]

utility_id
utility_name
state


[None, None, None]

I'm merging by utility_id so let's fix the state variable

In [7]:
gen_data = gen_data.rename(columns={'state': 'plant_state'})

In [8]:
merged = gen_data.merge(utils, how='left', on=['utility_id','utility_name'])

Various checks assures me that the merge was successful. So now I want to merge in ownership data in a clever way

## Now merge in the owner variable in a creative way
### solution is to add owner_1 owner_1_perc, owner_2 ... etc.  for each plant

In [9]:
# S Single ownership by respondent 
# J Jointly owned with another entity
# W Wholly owned by an entity other than respondent
df = merged.join(pd.get_dummies(merged['ownership']))
df.rename(columns={'S': 'single_owner', 'W': 'other_owner', 'J':'joint_owner'},inplace=True)

In [10]:
owners = pd.read_csv(dataDir + 'eia_data/eia8602018/owner_2018.csv',header=1)
owners.columns = owners.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('?','')

owners['perc_owned'] = pd.to_numeric(owners['percent_owned'].str.replace("%",""))

num_owners = owners.groupby(['plant_name', 'generator_id']).apply(lambda grp: len(grp.owner_name.unique())).reset_index()
num_owners.rename(columns={0: 'num_owners'},inplace=True)
owners = owners.merge(num_owners, how='outer', on=['plant_name','generator_id'])

In [11]:
owners['idx'] = owners['plant_code'].astype(str) + '_' + owners['generator_id']

In [12]:
# owner_dict with key = generator; value = owners 
owner_dict = owners.groupby(['idx']).owner_name.apply(lambda grp: list(grp.value_counts().index)).to_dict()

perc_dict = owners.groupby(['idx']).perc_owned.apply(lambda grp: list(grp)).to_dict()

# dictionary may be useful for other stuff 
perc_owned_dict = {}
for x in owner_dict:
    temp_dict = {}
    for i,y in enumerate(owner_dict[x]):
        temp_dict.update({y : perc_dict[x][i]})
    perc_owned_dict[x] = temp_dict

In [13]:
owners = owners.join(owners['idx'].map(owner_dict).apply(pd.Series).iloc[:,0:4].rename(columns={0: 'owner_1', 1: 'owner_2', 2: 'owner_3', 3: 'owner_4'}))

owners = owners.join(owners['idx'].map(perc_dict).apply(pd.Series).iloc[:,0:4].rename(columns={0: 'perc_owner_1', 1: 'perc_owner_2', 2: 'perc_owner_3', 3: 'perc_owner_4'}))


## Apply parent dict to the list of owners

In [14]:
with open(jsonDir + 'final_parent_dict.json') as outfile:
    parent_map = json.load(outfile)

In [15]:
owners['parent_1'] = owners.owner_1.str.lower().map(parent_map)
owners['parent_2'] = owners.owner_2.str.lower().map(parent_map)
owners['parent_3'] = owners.owner_3.str.lower().map(parent_map)
owners['parent_4'] = owners.owner_4.str.lower().map(parent_map)

In [16]:
owners['diff_parent'] = owners['parent_1'] != owners['owner_1']

## Drop duplicates before merging

In [17]:
# list of cols to ignore to avoid _x and _y 
cols_to_ignore = ['utility_id',
 'utility_name',
 'plant_name',
 'state',
 'status'] 

owners_to_merge = owners.drop_duplicates('idx')
owners_to_merge = owners_to_merge.drop([x for x in df.columns if x in owners.columns if x in cols_to_ignore],axis=1)
df = df.merge(owners_to_merge, how='left', on=['plant_code','generator_id'])

In [23]:
df.to_csv(dataDir + 'master_gen_by_owner_2028.csv')

## Merge owners back into generator data

### Now do the parent utility stuff

In [18]:
# fill na of parent 1 with parent map on the utility name

df['parent_1'] = df['parent_1'].fillna(df.utility_name.str.lower().map(parent_map))

In [19]:
df['nameplate_cap']=pd.to_numeric(df['nameplate_capacity_mw'].astype(str).str.replace(",",""))

In [20]:
solar = df[df['energy_source_1']=='SUN']
wind=df[df['energy_source_1']=='WND']

In [21]:
solar.groupby('plant_state').nameplate_cap.sum().sort_values(ascending=False).reset_index()

plant_state  nameplate_cap
0           CA        11837.6
1           NC         4007.9
2           AZ         2067.3
3           TX         1943.1
4           NV         1926.0
5           FL         1401.6
6           GA         1026.2
7           UT          859.1
8           NJ          771.9
9           MA          744.9
10          MN          741.5
11          NM          562.2
12          CO          532.6
13          VA          462.0
14          MD          355.2
15          SC          353.0
16          OR          312.8
17          NY          268.6
18          ID          240.0
19          IN          216.3
20          AL          196.9
21          TN          180.6
22          MS          160.6
23          HI          124.1
24          VT          100.6
25          AR          100.0
26          MI           99.3
27          WY           92.0
28          CT           86.1
29          OH           84.8
30          MO           62.1
31          PA           59.7
32          IL           40.9
33          DE           33.4
34          RI           31.6
35          OK           30.5
36          KY           26.3
37          WI           23.9
38          WA           19.7
39          NE           17.9
40          MT           17.0
41          IA            9.2
42          ME            5.6
43          KS            4.0
44          LA            1.1
45          SD            1.0

In [230]:
solar[solar['plant_state']=='TX'].groupby('parent_1').nameplate_cap.sum().sort_values(ascending=False).reset_index()

parent_1  nameplate_cap
0        consolidated edison development inc.          355.5
1                            southern company          218.5
2                             clearway energy          202.0
3                      174 power global corp.          182.0
4                    upton county solar 2 llc          180.0
5                                 re roserock          160.0
6                          berkshire hathaway          155.0
7                    cypress creek renewables          110.8
8                        solaireholman 1, llc           50.0
9                     lamesa solar ii, l.l.c.           50.0
10                            oci solar power           45.1
11                      fremont energy center           39.6
12               first solar asset management           30.2
13                        bluebell solar, llc           30.0
14                           frv ae solar llc           30.0
15                         phantom solar, llc           15.4
16                                duke energy           13.9
17                              sune cps3 llc           10.6
18                           newman solar llc           10.5
19           macquarie infrastructure company           10.0
20                              sune cps1 llc           10.0
21                              sune cps2 llc           10.0
22                           pgc plano i, llc            7.7
23                        el paso electric co            3.0
24                        powerfin asl 1, llc            2.6
25                          ikea property inc            2.6
26         renewable energy alternatives, llc            2.0
27                 ameresco federal solutions            2.0
28  harvest moon renewable energy company llc            1.6
29                         socore bandera llc            1.5
30                clean energy collective llc            1.0

In [228]:
wind

utility_id                                utility_name  plant_code  \
195         13642                  Nome Joint Utility Systems        90.0   
196         13642                  Nome Joint Utility Systems        90.0   
197         13642                  Nome Joint Utility Systems        90.0   
887         10633                        City of Lamar - (CO)       508.0   
888         10633                        City of Lamar - (CO)       508.0   
...           ...                                         ...         ...   
22006       58270                          GE Wind Energy LLC     62642.0   
22007       58270                          GE Wind Energy LLC     62642.0   
22076       62657                               Harbec Energy     62736.0   
22077       62657                               Harbec Energy     62736.0   
22087       62645  General Services Administration - Region 8     62782.0   

                                    plant_name plant_state   county  \
195                                Snake River          AK     Nome   
196                                Snake River          AK     Nome   
197                                Snake River          AK     Nome   
887                                Lamar Plant          CO  Prowers   
888                                Lamar Plant          CO  Prowers   
...                                        ...         ...      ...   
22006                               GE-Lubbock          TX  Lubbock   
22007                               GE-Lubbock          TX  Lubbock   
22076                            Harbec Energy          NY  Ontario   
22077                            Harbec Energy          NY  Ontario   
22087  Pembina Land Port of Entry Wind Turbine          ND  Pembina   

      generator_id            technology prime_mover unit_code ownership  \
195            ENT  Onshore Wind Turbine          WT       NaN         S   
196          EWT 1  Onshore Wind Turbine          WT       NaN         S   
197          EWT 2  Onshore Wind Turbine          WT       NaN         S   
887          T1-T3  Onshore Wind Turbine          WT       NaN         S   
888             T4  Onshore Wind Turbine          WT       NaN         W   
...            ...                   ...         ...       ...       ...   
22006        GE-P2  Onshore Wind Turbine          WT       NaN         S   
22007        GE-P3  Onshore Wind Turbine          WT       NaN         S   
22076        250KW  Onshore Wind Turbine          WT       NaN         S   
22077        850KW  Onshore Wind Turbine          WT       NaN         S   
22087        PEMWT  Onshore Wind Turbine          WT       NaN         S   

      duct_burners can_bypass_heat_recovery_steam_generator  \
195              X                                        X   
196              X                                        X   
197              X                                        X   
887              X                                        X   
888              X                                        X   
...            ...                                      ...   
22006            X                                        X   
22007            X                                        X   
22076            X                                        X   
22077            X                                        X   
22087            X                                        X   

      rto/iso_lmp_node_designation  \
195                            NaN   
196                            NaN   
197                            NaN   
887                            NaN   
888                            NaN   
...                            ...   
22006                          NaN   
22007                          NaN   
22076                          NaN   
22077                          NaN   
22087                          NaN   

      rto/iso_location_designation_for_reporting_wholesale_sales_data_to_ferc  \
195                  

## answer ga solar questions


In [25]:
ga_solar = solar[solar['plant_state']=='GA']


In [35]:
ga_solar[ga_solar['parent_1'] == 'southern company']

utility_id       utility_name  plant_code  \
19075       17650  Southern Power Co     59449.0   
19076       17650  Southern Power Co     59450.0   
19185        7140   Georgia Power Co     59556.0   
19552        7140   Georgia Power Co     59862.0   
19553        7140   Georgia Power Co     59863.0   
19554        7140   Georgia Power Co     59864.0   
19555        7140   Georgia Power Co     59865.0   
19563        7140   Georgia Power Co     59876.0   
19567       17650  Southern Power Co     59891.0   
19568       17650  Southern Power Co     59894.0   
19569       17650  Southern Power Co     59896.0   
19570       17650  Southern Power Co     59897.0   
20193        7140   Georgia Power Co     60544.0   
21291        7140   Georgia Power Co     61554.0   
21881        7140   Georgia Power Co     62393.0   
21882        7140   Georgia Power Co     62393.0   
21883        7140   Georgia Power Co     62393.0   

                               plant_name plant_state         county  \
19075        Decatur County Solar Project          GA        Decatur   
19076  Decatur Parkway Solar Project, LLC          GA        Decatur   
19185                            Dalton 2          GA         Murray   
19552         Fort Benning Solar Facility          GA  Chattahoochee   
19553          Fort Gordon Solar Facility          GA       Richmond   
19554           King's Bay Solar Facility          GA         Camden   
19555         Fort Stewart Solar Facility          GA        Liberty   
19563   Marine Corps Logistics Base Solar          GA      Dougherty   
19567                Butler Solar Farm 20          GA         Taylor   
19568                  Pawpaw Solar Plant          GA         Taylor   
19569            Butler Solar Project 103          GA         Taylor   
19570                 Taylor County Solar          GA         Taylor   
20193           Tri-County Solar Facility          GA         Putnam   
21291                         Comer Solar          GA        Madison   
21881               Atlanta Falcons Solar          GA         Fulton   
21882               Atlanta Falcons Solar          GA         Fulton   
21883               Atlanta Falcons Solar          GA         Fulton   

      generator_id          technology prime_mover unit_code ownership  \
19075        DCSP1  Solar Photovoltaic          PV       NaN         S   
19076        DPSP1  Solar Photovoltaic          PV       NaN         S   
19185            1  Solar Photovoltaic          PV       NaN         S   
19552            1  Solar Photovoltaic          PV       NaN         S   
19553            1  Solar Photovoltaic          PV       NaN         S   
19554            1  Solar Photovoltaic          PV       NaN         S   
19555            1  Solar Photovoltaic          PV       NaN         S   
19563            1  Solar Photovoltaic          PV       NaN         S   
19567            1  Solar Photovoltaic          PV       NaN         S   
19568            1  Solar Photovoltaic          PV       NaN         S   
19569            1  Solar Photovoltaic          PV       NaN         S   
19570            1  Solar Photovoltaic          PV       NaN         S   
20193          PV1  Solar Photovoltaic          PV       NaN         S   
21291            1  Solar Photovoltaic          PV       NaN         S   
21881            1  Solar Photovoltaic          PV       NaN         S   
21882            2  Solar Photovoltaic          PV       NaN         S   
21883            3  Solar Photovoltaic          PV       NaN         S   

      duct_burners can_bypass_heat_recovery_steam_generator  \
19075            X                                        X   
19076            X                                        X   
19185            X                                        X   
19552            X                                        X   
19553            X                                        X   
19554            X                                        X   